# Zipline Coding Exercises

In this notebook you will create your own pipeline and create custom factors and filters for it. 

### Install Packages

In [8]:
import sys

!{sys.executable} -m pip install --upgrade pip

!{sys.executable} -m pip install --upgrade setuptools build wheel

!{sys.executable} -m pip install -r requirements.txt

  Using cached SQLAlchemy-1.3.22-cp36-cp36m-manylinux2010_x86_64.whl (1.3 MB)
  Using cached numpy-1.19.1-cp36-cp36m-manylinux2010_x86_64.whl (14.5 MB)
  Using cached zipline-1.3.0.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached graphviz-0.9-py2.py3-none-any.whl (16 kB)
  Using cached Logbook-1.5.3.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached requests_file-1.5.1-py2.py3-none-any.whl (3.7 kB)
  Using cached pandas-0.22.0-cp36-cp36m-manylinux1_x86_64.whl (26.2 MB)
  Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)
  Using cached cyordereddict-1.0.0.tar.gz (138 kB)
  Preparing metadata (setup.py) ... done
  Using cached Bottleneck-1.3.5-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (351 kB)
  Using cached contextlib2-21.6.0-py2.py3-none-any.whl (13 kB)
  Using cached bcolz-0.12.1.tar.gz (622 kB)
  Preparing metadata (setup.py) ... done
  Using cached multipledispatch-0.6.0-p

  Created wheel for zipline: filename=zipline-1.3.0-cp36-cp36m-linux_x86_64.whl size=5212117 sha256=18960769d09926b60debe2d173b928fa7be1d31fc5c11c497405d242d28cdc1c
  Stored in directory: /root/.cache/pip/wheels/47/3e/96/5819c9453bd1cbe7cd0a9356df16744b1e2366e9b25071a9bb
  Created wheel for bcolz: filename=bcolz-0.12.1-cp36-cp36m-linux_x86_64.whl size=1112265 sha256=c0bd2edc615fce46fb3babd41387f03c75049337469da239a0e38a5ad7f63db2
  Stored in directory: /root/.cache/pip/wheels/9f/51/d6/173c1dabc3904530cd9527026946789e2a065b004916e5c5bb
  Created wheel for cyordereddict: filename=cyordereddict-1.0.0-cp36-cp36m-linux_x86_64.whl size=169570 sha256=c6d16c1eb634cb774b8af783ee640ec88015d15b9ddf816b90a0b94a9054fe32
  Stored in directory: /root/.cache/pip/wheels/8d/ff/1a/5f19b34a20e254f738ef53a8469e9e92ee13e66d54de3ea89c
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39780 sha256=ece8b363eb8998a741b1ed80a4ca26b31f2c7bdd622f3f21b070a6ff0d6a04b7
  Stored in director

In [9]:
!sed -Ei 's/^# deb-src /deb-src /' /etc/apt/sources.list 
!apt-get update 
!apt-get install libhdf5-dev -y

Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [99.8 kB]
Get:2 https://cli-assets.heroku.com/apt ./ InRelease [2,550 B]                 
Hit:4 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu xenial InRelease 
Hit:5 http://archive.ubuntu.com/ubuntu xenial InRelease                        
Hit:6 http://archive.canonical.com/ubuntu xenial InRelease               
Err:2 https://cli-assets.heroku.com/apt ./ InRelease                     
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 536F8F1DE80F6A35
Get:7 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [99.8 kB]
Hit:8 https://deb.nodesource.com/node_11.x xenial InRelease    
Get:9 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [97.4 kB]
Hit:3 https://downloads.apache.org/cassandra/debian 311x InRelease
Fetched 299 kB in 1s (269 kB/s)
Reading package lists... Done
W: An error occurred during the signature verification. The repository is not

In [10]:
# import sys
# !{sys.executable} -m pip install -r requirements.txt

# Load Data

In [11]:
import os

from zipline.data import bundles
from zipline.data.bundles.csvdir import csvdir_equities


# Specify the bundle name
bundle_name = 'm4-quiz-eod-quotemedia'

# Create an ingest function 
ingest_func = csvdir_equities(['daily'], bundle_name)

# Register the data bundle and its ingest function
bundles.register(bundle_name, ingest_func);

# Set environment variable 'ZIPLINE_ROOT' to the path where the most recent data is located
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..', 'data', 'module_4_quizzes_eod')

# Load the data bundle
bundle_data = bundles.load(bundle_name)

# TODO: Build an Empty Pipeline with a Screen

In the code below create an empty with a screen that filters the pipeline output for stocks with a 60-day average dollar volume greater than \$50,000.

In [17]:
# import resources
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume

# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length = 60) > 50000

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen = universe)

# TODO: Create a Pipeline Engine

In the code below, create a data loader that loads our data bundle using Zipline's `USEquityPricingLoader`. Then create a function to be passed to the `get_loader` parameter in the pipeline engine. Using the NYSE trading calendar create a pipeline engine using Zipline's `SimplePipelineEngine`

In [18]:
# import resources
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.loaders import USEquityPricingLoader

# Set the dataloader
pricing_loader = USEquityPricingLoader(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)

# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader

# Set the trading calendar
trading_calendar = get_calendar('NYSE')

# Create a Pipeline engine
engine = SimplePipelineEngine(get_loader = choose_loader,
                              calendar = trading_calendar.all_sessions,
                              asset_finder = bundle_data.asset_finder)

# TODO: Create Factors and Filters For The Pipeline

### Factors

In the code below, create two factors:

1. **Custom Factor**: Create a custom factor, `percent_difference`,  by combining a 30-day average closing price factor and a 60-average closing price factor. The `percent_difference` factor will calculate the difference between the 30-day and 60-average closing price factors, and will then normalize this difference by the 60-average closing price factor.


2. **Daily Returns Factor**: Create a factor that returns the daily percent change in closing price. To do this, use Zipline's built-in [DailyReturns](https://www.zipline.io/appendix.html#zipline.pipeline.factors.DailyReturns) factor.

Both factors should use the closing prices in the `USEquityPricing`  dataset as inputs. 




### Filters

In the code below, you will create a **Custom Filter**, `tradable_asset`, that returns `True` for the top 20 securities of average dollar volume in a 30 day-window that have a latest closing price above \$30. To do this, you will create two filters first and them combine them. For the first filter, you can use Zipline's built-in factor `AverageDollarVolume()` and its method `.top()` to select the top 20 assets of average dollar volume in a 30 day-window. To get the latest close price of the `USEquityPricing` dataset you can use `USEquityPricing.close.latest`. Finally you can combine these two filters by using the logical operator `&`. 

In [19]:
# import resources
import pandas as pd
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.factors import DailyReturns

# Create the 30-day average closing price factor
mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

# Create the 60-day average closing price factor
mean_close_60 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=60)

# Create the custom factor
percent_difference = (mean_close_30 - mean_close_60) / mean_close_60

# Create the daily returns factor
daily_ret = DailyReturns(inputs=[USEquityPricing.close])

# Create a filter for the top 20 securities of average dollar volume in a 30 day-window
top_20 = AverageDollarVolume(window_length=30).top(20)

# Create a filter for the latest closing price above $30
above_30 = USEquityPricing.close.latest > 30

# Create a custum filter
tradable_asset = top_20 & above_30


# TODO: Add Factors and Filters To The Pipeline

In the code below, add the factors and filters you created above to the pipeline

In [20]:
# Add the custom factor to the pipeline
pipeline.add(percent_difference, 'Percent Difference')

# Add the daily returns factor to the pipeline
pipeline.add(daily_ret, 'Daily Return')

# Add the custom filter to the pipeline
pipeline.add(tradable_asset, 'Tradable')

# TODO: Run The Pipeline

In the code below, run the pipeline for the dates given

In [21]:
# Set starting and end dates
start_date = pd.Timestamp('2014-01-06', tz='utc')
end_date = pd.Timestamp('2016-01-05', tz='utc')

# Run our pipeline for the given start and end dates
output = engine.run_pipeline(pipeline, start_date, end_date)

# Display the pipeline output
output.head()

Daily Return  Percent Difference  \
2014-01-06 00:00:00+00:00 Equity(0 [A])         0.006951            0.025939   
                          Equity(1 [AAL])       0.046523            0.055887   
                          Equity(2 [AAP])       0.028611            0.051009   
                          Equity(3 [AAPL])     -0.021972            0.035906   
                          Equity(4 [ABBV])      0.006147            0.033018   

                                            Tradable  
2014-01-06 00:00:00+00:00 Equity(0 [A])        False  
                          Equity(1 [AAL])      False  
                          Equity(2 [AAP])      False  
                          Equity(3 [AAPL])      True  
                          Equity(4 [ABBV])     False

## Solution

[Solution notebook](zipline_coding_exercises_solution.ipynb)